In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

import torch 
import torch.nn as nn

In [2]:
df = pd.read_csv("C:/Users/kzhan/Desktop/pytorchTutorial/datasets/star_classification.csv")

In [3]:
df.head(n=5)

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [4]:
train_df = df.drop(["obj_ID","run_ID","rerun_ID","field_ID","fiber_ID",
         "cam_col","alpha","delta"], axis = 1)

In [5]:
from sklearn.model_selection import train_test_split

unique_classes = np.unique(df["class"])

X, y = df.drop(["class"], axis = 1), df["class"]
y = y.map({"GALAXY": 0, "QSO": 1, "STAR": 2})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [6]:
X_train = torch.from_numpy(np.array(X_train)).float()
X_test = torch.from_numpy(np.array(X_test)).float()

y_train = torch.from_numpy(np.array(y_train)).long()
y_test = torch.from_numpy(np.array(y_test)).long()

In [7]:
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Classifier, self).__init__()
        self.linear_1 = nn.Linear(input_dim, hidden_dim)
        self.relu_1 = nn.ReLU()

        self.linear_2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu_2 = nn.ReLU()

        self.linear_3 = nn.Linear(hidden_dim, output_dim)
        self.relu_3 = nn.ReLU()
    
    def forward(self, x):
        outputs = self.relu_1(self.linear_1(x))
        outputs = self.relu_2(self.linear_2(outputs))
        outputs = self.relu_3(self.linear_3(outputs))
        return outputs 

In [8]:
n_samples, n_features = X_train.shape

model = Classifier(input_dim = n_features, hidden_dim = 10, output_dim = 3)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()

In [9]:
batch_size = 50000
n_iters = 3000
n_epochs = n_iters / (len(X_train) / batch_size)
n_epochs = int(n_epochs)
print(n_epochs)

1875


In [10]:
import tqdm

loss_list = torch.zeros((n_epochs, ))
accuracy_list = torch.zeros((n_epochs))

for epoch in tqdm.trange(n_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

100%|██████████| 1875/1875 [00:55<00:00, 33.68it/s]


In [11]:
accuracy_list

tensor([0.5930, 0.5930, 0.5930,  ..., 0.5930, 0.5930, 0.5930])